In [34]:
from typing import List
from itertools import accumulate
from math import sqrt, ceil
import numpy as np
import scipy.linalg as la
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import norm
import quimb.tensor as qtn
import cirq
import openfermion as of
from error_pert import (
    get_v2_sarray,
    get_v2_contri
)
from qpe_trotter import (
    group_single_strings,
    trotter_perturbation,
    get_gate_counts,
    sample_eps2
)
from tensor_network_common import mps_to_vector, pauli_sum_to_mpo

In [43]:
def commutator(a, b):
    return a * b - b * a

def v2_pauli_sum(hamiltonian_terms: List[cirq.PauliSum]) -> cirq.PauliSum:
    """"Calculates V2 the same way as in the get_v2_sarray function of error_pert.py"""
    
    nterms = len(hamiltonian_terms)
    term_sums_l2r = list(accumulate(hamiltonian_terms))
    temp = reversed(hamiltonian_terms)
    term_sums_r2l = list(accumulate(temp))
    term_sums_r2l = list(reversed(term_sums_r2l))
    term_sums_r2l.append(cirq.PauliSum())
    term_combs_V1_v2 = [(term_sums_l2r[i-1], hamiltonian_terms[i], term_sums_r2l[i+1]) for i in range (1, nterms)]
    v2 = cirq.PauliSum()
    for i,j,k in term_combs_V1_v2:
        V1_term = commutator(i, j)
        v2 += - commutator(V1_term, k)*1/3 - commutator(V1_term, j)*1/6
    return v2

In [27]:
energy_error = 1e-3

## Fermi-Hubbard

In [28]:
l1 = 2
l2 = 3
t, u = 1.0, 4.0
ham = of.fermi_hubbard(l1, l2, t, u, spinless=True)
ham_jw = of.transforms.jordan_wigner(ham)
nterms = len(ham_jw.terms)
print(f"Hamiltonian has {nterms} terms.")
ham_cirq = of.transforms.qubit_operator_to_pauli_sum(ham_jw)
qs = ham_cirq.qubits
nq = len(qs)
print(f"Hamiltonian has {nq} qubits.")

Hamiltonian has 34 terms.
Hamiltonian has 6 qubits.


In [29]:
ham_terms = [ps for ps in ham_cirq]

In [30]:
# Use the code from the paper.
ps_matrices = [ps.matrix(qs) for ps in ham_terms]
ps_sparse = [csc_matrix(psm) for psm in ps_matrices]
v2 = get_v2_sarray(ps_sparse)

In [31]:
# Get the ground state by DMRG and convert to a vector.
max_mps_bond = 20
max_mpo_bond = 100
ham_mpo = pauli_sum_to_mpo(ham_cirq, qs, max_mpo_bond)
dmrg = qtn.tensor_dmrg.DMRG(ham_mpo, max_mps_bond)
converged = dmrg.solve()
if not converged:
    print("DMRG did not converge!")
ground_state = dmrg.state
ground_energy = dmrg.energy.real
print(f"Final DMRG energy: {ground_energy:4.5e}")
gs_vector = mps_to_vector(ground_state)

Final DMRG energy: -3.12311e+00
sorted_inds = ['k0', 'k1', 'k2', 'k3', 'k4', 'k5']


/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: divide by zero encountered in matmul
  return func(*args, **kwargs)
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: overflow encountered in matmul
  return func(*args, **kwargs)
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: invalid value encountered in matmul
  return func(*args, **kwargs)


In [32]:
eps2 = np.vdot(gs_vector, v2 @ gs_vector).real
print(f"eps2 = {eps2:4.5e}")
dt = sqrt(energy_error / eps2)
print(f"dt = {dt:4.5e}")

eps2 = 2.35403e+01
dt = 6.51770e-03


In [33]:
# Get the evolution time.
if nq <= 10:
    ham_sparse = of.linalg.get_sparse_operator(ham_jw)
    ham_norm = norm(ham_sparse)
else:
    # Approximate the norm of the Hamiltonian with the triangle inequality.
    # This is an upper bond on the norm, so we will have smaller tau than we should.
    coeffs = np.array([ps.coefficient for ps in ham_cirq])
    ham_norm = np.sum(np.abs(coeffs))
evol_time = np.pi / (4. * ham_norm)
print(f"Evolution time = {evol_time}")
nsteps = ceil(evol_time / dt)
print(f"Needs {nsteps} Trotter step(s).")

Evolution time = 0.008056319245418457
Needs 2 Trotter step(s).


In [49]:
# Now get eps2 with my subroutine.
terms = [ps for ps in ham_cirq]
v2_me = v2_pauli_sum(terms)
qubit_map = {q: i for i, q in enumerate(qs)}
eps2_me = v2_me.expectation_from_state_vector(gs_vector, qubit_map).real
print(f"My eps2 = {eps2_me:4.5e}")
eps2_error = abs(eps2_me - eps2)
print(f"Absolute error {eps2_error:4.5e}")

My eps2 = 2.35403e+01
Absolute error 3.55271e-15


## $H_2$ molecule